In [1]:
import torch
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print((itos))

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [26]:
def train_mlp(hyperparams):
    def build_dataset(words):
        X, Y = [], []

        for w in words:
            context = [0] * hyperparams['block_size']
            for ch in w + '.':
                ix = stoi[ch]
                
                X.append(context)
                Y.append(ix)

                context = context[1:] + [ix] # crop and append 
        
        X = torch.tensor(X)
        Y = torch.tensor(Y)

        return X, Y

    random.seed(42)
    random.shuffle(words)
    n1 = int(0.8 * len(words))
    n2 = int(0.9 * len(words))

    Xtr, Ytr = build_dataset(words[:n1])
    Xdev, Ydev = build_dataset(words[n1:n2])
    Xte, Yte = build_dataset(words[n2:])

    g = torch.Generator().manual_seed(2147483647)
    C = torch.randn((hyperparams['characters'], hyperparams['input_dimensions']), generator=g)
    W1 = torch.randn((hyperparams['input_dimensions'] * hyperparams['block_size'], hyperparams['neurons']), generator=g) * 0.01
    b1 = torch.randn((hyperparams['neurons']), generator=g) * 0
    W2 = torch.randn((hyperparams['neurons'], hyperparams['characters']), generator=g) * 0.01
    b2 = torch.randn((hyperparams['characters']), generator=g) * 0
    parameters = [C, W1, b1, W2, b2]

    for p in parameters:
        p.requires_grad = True

    sum(p.nelement() for p in parameters) # number of total params

    for i in range(200_000):
        # minibatch construct
        ix = torch.randint(0, Xtr.shape[0], (hyperparams['mini_batch_size'],))

        # forward
        emb = C[Xtr[ix]]
        h = torch.tanh(emb.view((-1, hyperparams['input_dimensions'] * hyperparams['block_size'])) @ W1 + b1)
        logits = h @ W2 + b2
        loss = F.cross_entropy(logits, Ytr[ix])

        # backward pass
        for p in parameters:
            p.grad = None
        loss.backward()

        # update
        lr = 0.1 if i < 100_000 else 0.01
        for p in parameters:
            p.data += -lr * p.grad
    
    def calculate_loss(inputs, outputs):
        emb = C[inputs]
        h = torch.tanh(emb.view((-1, hyperparams['input_dimensions'] * hyperparams['block_size'])) @ W1 + b1)
        logits = h @ W2 + b2
        return F.cross_entropy(logits, outputs)

    tr_loss = calculate_loss(Xtr, Ytr)
    dev_loss = calculate_loss(Xdev, Ydev)

    return tr_loss, dev_loss

In [28]:
experiments = [
    # {
    #     "block_size": 3,
    #     "input_dimensions": 10,
    #     "neurons": 200,
    #     "mini_batch_size": 48,
    #     "characters": 27,
    # },
    # {
    #     "block_size": 5,
    #     "input_dimensions": 10,
    #     "neurons": 200,
    #     "mini_batch_size": 48,
    #     "characters": 27,
    # },
    # {
    #     "block_size": 3,
    #     "input_dimensions": 10,
    #     "neurons": 250,
    #     "mini_batch_size": 48,
    #     "characters": 27,
    # },
    {
        "block_size": 8,
        "input_dimensions": 50,
        "neurons": 700,
        "mini_batch_size": 128,
        "characters": 27,
    }
]

losses = []
for experiment in experiments:
    print("Running experiment with params:", experiment)
    tr_loss, dev_loss = train_mlp(experiment)
    losses.append(dev_loss)
    print(f"Achieved loss of {tr_loss} and {dev_loss}")
    print('----------------------------')

min(losses)

# Running experiment with params: {'block_size': 3, 'input_dimensions': 10, 'neurons': 200, 'mini_batch_size': 48, 'characters': 27}
# Achieved loss of 2.1132707595825195 and 2.149827241897583
# --> base model
# ----------------------------
# Running experiment with params: {'block_size': 5, 'input_dimensions': 10, 'neurons': 200, 'mini_batch_size': 48, 'characters': 27}
# Achieved loss of 2.178300142288208 and 2.205777883529663
# --> more block_size doesn't help
# ----------------------------
# Running experiment with params: {'block_size': 5, 'input_dimensions': 10, 'neurons': 250, 'mini_batch_size': 48, 'characters': 27}
# Achieved loss of 2.166443347930908 and 2.2088193893432617
# ----------------------------
# Running experiment with params: {'block_size': 3, 'input_dimensions': 10, 'neurons': 250, 'mini_batch_size': 48, 'characters': 27}
# Achieved loss of 2.117893695831299 and 2.1639604568481445
# ----------------------------
# Running experiment with params: {'block_size': 3, 'input_dimensions': 20, 'neurons': 250, 'mini_batch_size': 48, 'characters': 27}
# Achieved loss of 2.0417332649230957 and 2.145442008972168
# ----------------------------
# Running experiment with params: {'block_size': 4, 'input_dimensions': 30, 'neurons': 300, 'mini_batch_size': 64, 'characters': 27}
# Achieved loss of 1.8625956773757935 and 2.044607162475586
# ----------------------------
# Running experiment with params: {'block_size': 8, 'input_dimensions': 50, 'neurons': 700, 'mini_batch_size': 128, 'characters': 27}
# Achieved loss of 1.6185532808303833 and 2.1509552001953125
# ----------------------------

Running experiment with params: {'block_size': 8, 'input_dimensions': 50, 'neurons': 700, 'mini_batch_size': 128, 'characters': 27}
Achieved loss of 1.6185532808303833 and 2.1509552001953125
----------------------------


tensor(2.1510, grad_fn=<NllLossBackward0>)

In [ ]:
for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))